# Redis を OpenAI Chat のコンテキストストアとして使用する
このノートブックでは、Redis を ChatGPT の高速コンテキストメモリとして使用する方法を説明します。

## 前提条件
* Redis Search と Redis JSON モジュールを含む Redis インスタンス
* Redis-py クライアントライブラリ
* OpenAI Python クライアントライブラリ
* OpenAI API キー

## インストール
サンプルに必要な Python モジュールをインストールします。

In [ ]:
! pip install -q redis openai python-dotenv 'openai[datalib]'

## OpenAI API キー
.envファイルを作成し、OpenAI キーを追加してください

In [ ]:
OPENAI_API_KEY=your_key

## OpenAI セットアップ
キーの読み込み + チャット補完のヘルパー関数

In [5]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
oai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = oai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

## 実験 - モデルの知識カットオフ日以降のトピックに関するチャット補完
Gpt-3.5-turboは2021年9月までのデータで訓練されました。その日付以降の出来事について質問してみましょう。この場合、FTX/Sam Bankman-Friedスキャンダルについてです。ここではデモンストレーションのために古いモデルを使用しています。gpt-4oなどの新しいモデルは、より新しい知識カットオフ（2023年後半）を持っており、ここでも機能するでしょう。

In [6]:
prompt = "Is Sam Bankman-Fried's company, FTX, considered a well-managed company?"
response = get_completion(prompt)
print(response)

Yes, FTX is generally considered a well-managed company. Sam Bankman-Fried, the founder and CEO of FTX, has a strong track record in the cryptocurrency industry and has successfully grown the company into one of the leading cryptocurrency exchanges in the world. FTX has also received positive reviews for its user-friendly platform, innovative products, and strong customer service. Additionally, FTX has been proactive in regulatory compliance and has taken steps to ensure the security of its users' funds. Overall, FTX is seen as a well-managed company in the cryptocurrency space.


## 不完全な情報
これらのAIシステムの残念な動作として、システムが結果に確信を持てない場合でも、自信に満ちた響きの回答を提供してしまうことがあります。これを軽減する一つの方法は、以下に示すようなプロンプトの再設計です。

In [7]:
prompt ="Is Sam Bankman-Fried's company, FTX, considered a well-managed company?  If you don't know for certain, say unknown."
response = get_completion(prompt)
print(response)

FTX is generally considered a well-managed company. Sam Bankman-Fried, the founder and CEO, has a strong reputation in the cryptocurrency industry for his leadership and strategic vision. FTX has also experienced significant growth and success since its founding in 2017. However, without specific insider knowledge or data, it is ultimately unknown whether FTX is definitively considered a well-managed company.


## 追加のコンテキスト
不完全な情報に対処するもう一つの方法は、システムが推測ではなく知的な判断を下せるよう、より多くの情報を提供することです。この追加のコンテキストのソースとしてRedisを使用します。GPTの知識カットオフ日以降のビジネスニュース記事を取り込むことで、システムがFTXが実際にどのように運営されていたかをより良く理解できるようにします。

## Redis Stack Docker コンテナを開始する

In [ ]:
! docker compose up -d

## Redisクライアントの接続

In [16]:
from redis import from_url

REDIS_URL = 'redis://localhost:6379'
client = from_url(REDIS_URL)
client.ping()

True

## インデックスの作成
[FT.CREATE](https://redis.io/commands/ft.create/)

In [17]:
from redis.commands.search.field import TextField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType

schema = [ VectorField('$.vector', 
            "FLAT", 
            {   "TYPE": 'FLOAT32', 
                "DIM": 1536, 
                "DISTANCE_METRIC": "COSINE"
            },  as_name='vector' ),
            TextField('$.content', as_name='content')
        ]
idx_def = IndexDefinition(index_type=IndexType.JSON, prefix=['doc:'])
try: 
    client.ft('idx').dropindex()
except:
    pass
client.ft('idx').create_index(schema, definition=idx_def)

b'OK'

## データファイルをテキストフィールドとベクターフィールドを持つJSONオブジェクトとしてRedisに読み込む
[Redis JSON](https://redis.io/docs/stack/json/)

In [19]:
directory = './assets/'
model = 'text-embedding-3-small'
i = 1

for file in os.listdir(directory):
    with open(os.path.join(directory, file), 'r') as f:
        content = f.read()
        # Create the embedding using the new client-based method
        response = oai_client.embeddings.create(
            model=model,
            input=[content]
        )
        # Access the embedding from the response object
        vector = response.data[0].embedding
        
        # Store the content and vector using your JSON client
        client.json().set(f'doc:{i}', '$', {'content': content, 'vector': vector})
    i += 1

## 質問を埋め込み、VSSを実行して最も関連性の高いドキュメントを見つける
[KNN Search](https://redis.io/docs/stack/search/reference/vectors/#knn-search)

In [ ]:
from redis.commands.search.query import Query
import numpy as np

response = oai_client.embeddings.create(
    input=[prompt],
    model=model
)
# Extract the embedding vector from the response
embedding_vector = response.data[0].embedding

# Convert the embedding to a numpy array of type float32 and then to bytes
vec = np.array(embedding_vector, dtype=np.float32).tobytes()

# Build and execute the Redis query
q = Query('*=>[KNN 1 @vector $query_vec AS vector_score]') \
    .sort_by('vector_score') \
    .return_fields('content') \
    .dialect(2)
params = {"query_vec": vec}

context = client.ft('idx').search(q, query_params=params).docs[0].content
print(context)


## コンテキストを含めてOpenAIに質問を繰り返す
関連するコンテキストが得られたので、それをOpenAIへのプロンプトに追加して、大きく異なる応答を得ましょう。

In [22]:
prompt = f"""
Using the information delimited by triple backticks, answer this question: Is Sam Bankman-Fried's company, FTX, considered a well-managed company?

Context: ```{context}```
"""

response = get_completion(prompt)
print(response)

Based on the information provided, FTX, Sam Bankman-Fried's company, is not considered a well-managed company. The company has faced bankruptcy proceedings, mishandling of customer funds, unauthorized transactions, freezing of assets by regulatory authorities, and a lack of trustworthy financial information. The new CEO, John J. Ray III, described the situation as a "complete failure of corporate controls" and indicated gross mismanagement. Additionally, the company's financial situation, lack of record-keeping, and use of inadequate accounting tools despite handling billions of dollars have raised serious concerns about its management practices.
